# MSCF 46982 Market Microstructure and Algorithmic Trading

Fall 2025 Mini 2

Price Impact of Order Book Events

Copyright &copy; 2025 Nick Psaris. All Rights Reserved

# TOC
- [Initialize](#Initialize)
- [The Price Impact of Order Book Events](#The-Price-Impact-of-Order-Book-Events)
- [Order Book Events](#Order-Book-Events)
- [Order Flow Imbalance](#Order-Flow-Imbalance)
- [Market Depth](#Market-Depth)
- [Calculating the Impact of Order Flow Imbalance](#Calculating-the-Impact-of-Order-Flow-Imbalance)
- [Modeling the Price Impact of Order Flow Based on Market Depth](#Modeling-the-Price-Impact-of-Order-Flow-Based-on-Market-Depth)

# Initialize
- We start by initializing the number of rows and columns displayed
- Then create connection to the taq Kdb+ server
- And finally define a few utility functions (rnd, pivot and ols)


In [1]:
import os
os.environ['PYKX_JUPYTERQ'] = 'true'
os.environ['PYKX_4_1_ENABLED'] = 'true'
import pykx as kx


PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


In [2]:
\c 15 120
/ windows and mac/linux use different environment variables
home:`HOME`USERPROFILE "w"=first string .z.o
upf:0N!` sv (hsym`$getenv home),`cmu_userpass.txt
h:`$":tcps://tpr-mscf-kx.tepper.cmu.edu:5000:",first read0 upf
rnd:{x*"j"$y%x}
pivot:{[t]
 u:`$string asc distinct last f:flip key t;
 pf:{x#(`$string y)!z};
 p:?[t;();g!g:-1_ k;(pf;`u;last k:key f;last key flip value t)];
 p}
ols:{[i;Y;X]
 if[type Y;:.z.s[i;enlist Y;X]];
 if[type X;:.z.s[i;Y;enlist X]];
 if[i;X:enlist[count[X 0]#1f],X];
 THETA:Y lsq X;
 THETA}

`:/Users/nick/cmu_userpass.txt


# The Price Impact of Order Book Events

Cont, Rama and Kukanov, Arseniy and Stoikov, Sasha, [The Price Impact
of Order Book Events][] (April 30, 2012). JOURNAL OF FINANCIAL
ECONOMETRICS (Winter 2014) 12 (1): 47-88.

[The Price Impact of Order Book Events]: http://dx.doi.org/10.2139/ssrn.1712822 "The Price Impact of Order Book Events"

- Rama Cont, Arseniy Kukanov, Sasha Stoikov demonstrate that over
  short time intervals price changes are driven by order flow
  imbalances (OFI)
- The price impact of order flow encompasses the price impact of trades
  because trades impact the order book and decrease supply
- Cont et al. provide a model that depends on **average depth** which
  is defined as average sum of bid/ask sizes
- In principle, there should be more information in order book events
  because there are many more of them
- In practice, the importance of order book events depends on how
  **reliable** they are
- [Some have speculated][] that order flow imbalances have no
  explanatory power in the cryptcurrency markets because there are
  **no regulations that prevent spoofing**

[Some have speculated]: https://medium.com/@eliquinox/order-flow-analysis-of-cryptocurrency-markets-b479a0216ad8

 

# Order Book Events

- The order flow contribution for a single event can be described by
changes in bid/ask prices and sizes
$$
e_n = I_{P_n^B\ge P_{n-1}^B}q_n^B 
    - I_{P_n^B\le P_{n-1}^B}q_{n-1}^B
    - I_{P_n^A\le P_{n-1}^A}q_n^BA 
    + I_{P_n^A\ge P_{n-1}^A}q_{n-1}^A
$$
where $I$ is an indicator to determine the state, $P^B$ and $P^A$ are
the bid/ask prices, $q^B$ and $q^A$ are the bid/ask sizes and $n$ is
the current state while $n-1$ is the previous state

- **Increases** in bid prices or sizes indicate an **increases** in
  demand
- **Decreases** in bid prices or sizes indicate a **decreases** in
  demand
- **Increases** in ask prices or **decreases** in ask sizes indicate a
  **decrease** in supply
- **Decreases** in ask prices or **increases** in ask sizes indicates an
  **increases** in supply


In [3]:
ofi:{[b;bs;a;as]
 e:               bs *b>=pb:prev[first b;b];
 e-:prev[first bs;bs]*b<=pb;
 e-:              as *a<=pa:prev[first a;a];
 e+:prev[first as;as]*a>=pa;
 e}

# Order Flow Imbalance

- The order flow imbalance is the cumulative effect of all events
  between time $t_k-1$ and $t_k$
$$
\text{OFI}_k = \sum_{n=N(t_{k-1})+1}^{N(t_k)} e_n
$$
where $N(t_k)$ is the Nth order book event corresponding to the time $t_k$

- To describe the impact on prices, Cont et al. define the price change (in ticks) between time $t_k-1$ and $t_k$ as
$$
\Delta P_k = (P_k - P_{k-1})/\delta
$$
where $\delta$ is the tick size
- The model specifies a simple linear relationship between $\Delta
  P_k$ and $\text{OFI}_k$
$$
\Delta P_k = \alpha + \beta \hspace{0.5em} \text{OFI}_k + \epsilon_k
$$

# Market Depth

- Cont et al. also define 'market depth' as the average size of the
  best bid and offer quote sizes (which we will use later)
$$
\text{AD}_i = \frac{1}{2N(T_i) - N(T_{i-1})-1} \sum_{n=N(T_{i-1})+1}^{N(T_i)} (q_n^B + q_n^A)
$$

# Calculating the Impact of Order Flow Imbalance

- To remove abnormal trading behavior, Cont et al. exclude the first
  and last 30 minutes of trading
- The implementation computes the order flow imbalance (OFI) every 10
  seconds


## Impact of order flow imbalance results

- Regressions are performed every 30 minutes and then averaged
- We can see how low (high) priced securities move a small (large)
  amount for changes in the order flow
- We use notional (not price) to perform a cross-security comparisons
- Observe, also, that this is a **contemporaneous model**, and does
  not attempt to make predictions on future price changes based on
  current order flow imbalances


In [4]:
orderflow:{[ofif;dt;syms]
 t:select from nbbo where date = dt, sym in syms, time within 0D10 0D15:30, asize>0,bsize>0;
 t:update mid:.5*bid+ask,ofi:ofif[bid;bsize;ask;asize] by sym from t;
 s:select o:first mid, c:last mid, sum ofi, ad:avg .5*bsize+asize by sym,0D00:00:10 xbar time from t;
 s:update dticks:100*deltas[first o;c] by sym from s;
 s}
s:h (orderflow;ofi;2020.02.03;`AMZN`TSLA`NFLX`ORCL`CSCO`MU`BAC)

In [5]:
asc avg pivot b:select avg ad, beta:last last ols[1b;dticks;"f"$ofi] by 0D00:30 xbar time, sym from s

BAC | 0.0005664369
CSCO| 0.004715658
ORCL| 0.00481793
MU  | 0.008105073
NFLX| 0.4398638
TSLA| 1.169921
AMZN| 2.773596


# Modeling the Price Impact of Order Flow Based on Market Depth

- Using the average depth (AD), a relationship is found between the
  price impact coefficeint $\beta$ and the market depth $AD$
$$
\beta_i = \frac{c}{\text{AD}_i^\lambda }+ \nu_i
$$
where $c$ and $\lambda$ are constants and $\nu_i$ is a noise term

## Fit $\lambda$ coefficient

- $\lambda$ is fit first with a nonlinear regression by using linear
  regression on the **log values**
$$
\log \hat{\beta_i} = \hat{\alpha}_{L,i} - \hat{\lambda}\log\text{AD}_i + \hat{\epsilon}_{L,i}
$$

In [6]:
show l:select lambda:last last ols[1b;log beta;neg log ad] by sym from b

sym | lambda   
----| ---------
AMZN| 1.051458 
BAC | 1.399238 
CSCO| 1.471654 
MU  | 1.045026 
NFLX| 0.2949058
ORCL| 0.9505975
TSLA| 0.2956302


## Fit $c$ coefficient

- $c$ is then estimated for each symbol with simple linear regression
$$
\hat{\beta_i} = \hat{\alpha}_{M,i} + \frac{\hat{c}}{\text{AD}_i^{\hat{\lambda}}} + \hat{\epsilon}_{M,i}
$$

In [7]:
show c:select c:last last ols[1b;beta;1f%ad xexp lambda] by sym from  b lj l

sym | c        
----| ---------
AMZN| 5.528774 
BAC | 3.347822 
CSCO| 1.83946  
MU  | 0.2788601
NFLX| 0.4830386
ORCL| 0.2913703
TSLA| 1.146609 


## Compute $\hat{\beta}$

- Using $c$, $\lambda$ and the current market depth (AD), we can now
  estimate the price impact of order flow imbalance (OFI)
- What would the impact (in ticks) be of a single order at the bid
  when we have typical market depth?


In [8]:
beta:{[lambda;c;ad] c%ad xexp lambda}

In [9]:
t:l lj c lj select avg ad by sym from b
`dticks xasc update dticks:beta[lambda;c;ad] from t

sym | lambda    c         ad       dticks      
----| -----------------------------------------
BAC | 1.399238  3.347822  498.5888 0.0005623118
ORCL| 0.9505975 0.2913703 82.83719 0.00437507  
CSCO| 1.471654  1.83946   57.11529 0.004779232 
MU  | 1.045026  0.2788601 28.66863 0.008362934 
NFLX| 0.2949058 0.4830386 2.745448 0.3586182   
TSLA| 0.2956302 1.146609  3.395883 0.7988216   
AMZN| 1.051458  5.528774  1.972146 2.707152    


# Less-than-average depth

- What would the impact (in ticks) be of a single order at the bid
  when we have less-than typical market depth?


In [10]:
`dticks xasc update dticks:beta[lambda;c;.1*ad] from t

sym | lambda    c         ad       dticks    
----| ---------------------------------------
BAC | 1.399238  3.347822  498.5888 0.01409989
ORCL| 0.9505975 0.2913703 82.83719 0.03904654
MU  | 1.045026  0.2788601 28.66863 0.09276506
CSCO| 1.471654  1.83946   57.11529 0.1415834 
NFLX| 0.2949058 0.4830386 2.745448 0.7071932 
TSLA| 0.2956302 1.146609  3.395883 1.577902  
AMZN| 1.051458  5.528774  1.972146 30.47691  
